**Autores**: 
1. Caio Fernandes - 170138798
2. Lucas Dutras - 170050939

Este projeto tem como objetivo utilizar técnicas de data science juntamente com o conteúdo de **Percorrer Grafos** para indicar qual o caminho mais rápido para chegar em um país a partir de um país inicial.

Utilizaremos para isso dois datasets disponibilizados no Kaggle com informações sobre [Países e Fronteiras](https://www.kaggle.com/resheto/country-borders) e [Países e Continentes](https://www.kaggle.com/tomvebrcz/countriesandcontinents).

In [133]:
import pandas as pd

In [134]:
df_border = pd.read_csv('./data/border_info.csv')
df_continents = pd.read_csv('./data/countries.csv')
df_border = df_border.sort_values(by=['country_name'])
df_continents = df_continents.sort_values(by=['Country'])

## Avaliação do Dado

In [135]:
df_border.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 728 entries, 9 to 727
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           728 non-null    int64 
 1   country_name         728 non-null    object
 2   country_border_name  642 non-null    object
dtypes: int64(1), object(2)
memory usage: 22.8+ KB


## Limpando o dado 

Removendo países sem fronteira

In [136]:
is_NaN = df_border.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = df_border[row_has_NaN]
rows_with_NaN

,Unnamed: 0,country_name,country_border_name
41,41,Aland Islands,NaN
30,30,American Samoa,NaN
11,11,Anguilla,NaN
24,24,Antarctica,NaN
10,10,Antigua and Barbuda,NaN
...,...,...,...
684,684,United States Minor Outlying Islands,NaN
704,704,Vanuatu,NaN
699,699,Virgin Islands (British),NaN
700,700,Virgin Islands (U.S.),NaN


Temos 86 países sem fronteiras no dado

Removendo-os:

In [137]:
df_border = df_border.drop(rows_with_NaN.index)

Notamos que, neste dado em questão, temos 727 valores.
Tal dado também apresenta duas colunas, uma `country_name` representando o país e outra chamada `coutry_border_name` que representa um fronteira daquele país.

In [138]:
df_border.head()

,Unnamed: 0,country_name,country_border_name
9,9,Afghanistan,Uzbekistan
4,4,Afghanistan,China
5,5,Afghanistan,Iran
6,6,Afghanistan,Pakistan
7,7,Afghanistan,Tajikistan


Logo percebemos um problema, a maneira como o dado está organizado, nos trás informações duplicadas sobre países.
Para corrigir este problema, faremos, para cada país, uma coluna com uma lista de todas as fronteiras.
Com isso, teremos na estrutura do `DataFrame`, países únicos na coluna `country_name` com suas fronteiras listada na coluna `border_list`.

In [139]:
k = df_border.groupby('country_name')['country_border_name'].apply(lambda x: list(x))
df_border = df_border.drop(columns=['country_border_name'])
df_border = df_border.drop_duplicates(subset=['country_name'])
aux = [item for item in k]
df_border['border_list'] = aux

In [141]:
df_border = df_border.reset_index()
df_border = df_border.drop(columns=['Unnamed: 0', 'index'])
df_continents['Region'] = df_continents['Region'].replace('LATIN', 'AMERICA')
df_continents['Region'] = df_continents['Region'].replace('NORTHERN_AMERICA', 'AMERICA')
df_continents['Region'] = df_continents['Region'].replace('NEAR_EAST', 'ASIA')

In [142]:
border_l = df_border['country_name'].tolist()
cont_l = df_continents['Country'].tolist()
diff = []
for item in cont_l:
    if item in border_l:
        continent = df_continents.loc[df_continents['Country'] == item]
        continent = continent['Region']
        for elem in continent.iteritems():
            df_border.loc[df_border['country_name'] == item, 'continent'] = elem[1]


In [143]:
df_border.head(20)

,country_name,border_list,continent
0,Afghanistan,"[Uzbekistan, China, Iran, Pakistan, Tajikistan...",ASIA
1,Albania,"[Serbia, North Macedonia, Montenegro, Greece]",EUROPE
2,Algeria,"[Mali, Tunisia, Niger, Libya, Morocco, Western...",AFRICA
3,Andorra,"[France, Spain]",EUROPE
4,Angola,"[Congo (Kinshasa), Congo (the Democratic Repub...",AFRICA
5,Argentina,"[Uruguay, Brazil, Bolivia, Chile, Paraguay]",AMERICA
6,Armenia,"[Georgia, Iran, Turkey, Azerbaijan]",EUROPE
7,Austria,"[Slovenia, Czechia, Germany, Hungary, Liechten...",EUROPE
8,Azerbaijan,"[Georgia, Turkey, Armenia, Iran, Russia]",EUROPE
9,Bangladesh,"[Myanmar, India]",ASIA


In [144]:
df_continents['Region'].value_counts()

AFRICA     57
EUROPE     50
AMERICA    50
ASIA       49
OCEANIA    21
Name: Region, dtype: int64